In [103]:
import pandas as pd
Data = pd.read_csv('C:/Users/muneeba/Desktop/THESIS/DataDistribution/Gaussian3/gauss3/DataDistributionPCA.csv')
Data=Data.sort_values(['Sensor ID'],ascending=[True])
Data1=Data.iloc[:,4:68]
Data1.head()
returns = Data1[[key for key in dict(Data1.dtypes) if dict(Data1.dtypes)[key] in ['float64', 'int64']]] 
returns = returns[:64]
returns.head()
#slicing the required data out of complete dataset
returns_arr = returns.as_matrix()[:64]
returns_arr.shape
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
returns_arr_scaled = scaler.fit_transform(returns_arr)

In [104]:
import tensorflow as tf

n_inputs = 64
n_hidden = 3  # codings
n_outputs = n_inputs

learning_rate = 0.01 #how much to train 
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
hidden = tf.layers.dense(X, n_hidden)
outputs = tf.layers.dense(hidden, n_outputs)

reconstruction_loss = tf.reduce_mean(tf.square(outputs - X)) 

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(reconstruction_loss)

init = tf.global_variables_initializer()

In [105]:
n_iterations = 10000

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        training_op.run(feed_dict={X: returns_arr_scaled})
    writer = tf.summary.FileWriter('./temp', sess.graph)
    outputs_val = outputs.eval(feed_dict={X: returns_arr_scaled})
    print(outputs_val)

[[ 0.97646505  0.98959571  1.02127087 ...,  0.80938983  0.62447792
   0.44307074]
 [ 0.67687857  0.6877057   0.71406752 ...,  0.75257695  0.60476679
   0.45974866]
 [-0.31328174 -0.30999893 -0.30099669 ...,  0.57139838  0.54568881
   0.52039421]
 ..., 
 [-0.21821724 -0.21467389 -0.19581226 ...,  0.1633741   0.2235947
   0.27128011]
 [ 0.98923868  1.00098765  1.05803454 ..., -0.50587857 -0.3905969
  -0.31243989]
 [ 1.20362473  1.21676946  1.28035152 ..., -0.62797838 -0.50309443
  -0.41959062]]


In [106]:
import numpy as np
data=scaler.inverse_transform(outputs_val,copy=None)
#recovered
#data
orig=np.linalg.norm(outputs_val,ord=1)
DiffData=data-returns_arr
NewNorm=np.linalg.norm(DiffData,ord=1)
error=NewNorm/orig 
error
#recovering the orignal dat

0.00016048222236116807

In [107]:
#building stacked autoencoders
n_inputs = 64
n_hidden1 = 14
n_hidden2 = 3 # codings
n_hidden3 = n_hidden1
n_outputs = n_inputs

In [108]:
he_init = tf.contrib.layers.variance_scaling_initializer()
l2_regularizer = tf.contrib.layers.l2_regularizer(0.0001)

In [109]:
X = tf.placeholder(tf.float32, shape=[None, n_inputs])

In [110]:
from functools import partial

dense_layer = partial(tf.layers.dense,
                      activation=tf.nn.elu,
                      kernel_initializer=he_init,
                      kernel_regularizer=l2_regularizer)

hidden1 = dense_layer(X, n_hidden1)
hidden2 = dense_layer(hidden1, n_hidden2)
hidden3 = dense_layer(hidden2, n_hidden3)
outputs = dense_layer(hidden3, n_outputs, activation=None)
print(hidden2)

Tensor("dense_3/Elu:0", shape=(?, 3), dtype=float32)


In [111]:
reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))

reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([reconstruction_loss] + reg_losses)

optimizer = tf.train.AdamOptimizer(0.01)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [112]:
saver = tf.train.Saver()

In [101]:
n_iterations = 10000


with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        training_op.run(feed_dict={X: returns_arr_scaled})
    writer = tf.summary.FileWriter('./temp', sess.graph)
    outputs_val = outputs.eval(feed_dict={X: returns_arr_scaled})
    print(outputs_val)

[[ 0.97510237  0.98859227  1.02045476 ...,  0.82185435  0.63759363
   0.45602459]
 [ 0.67455405  0.68610823  0.71221483 ...,  0.76455736  0.61748385
   0.47219363]
 [-0.3159892  -0.31208152 -0.30367884 ...,  0.57494164  0.54992139
   0.52451527]
 ..., 
 [-0.21899891 -0.21557105 -0.19681245 ...,  0.16053867  0.22092414
   0.2687633 ]
 [ 0.99704486  1.00886977  1.06569862 ..., -0.5154829  -0.3998639
  -0.32153213]
 [ 1.21306443  1.22628784  1.28970551 ..., -0.63758326 -0.51248842
  -0.42892909]]


In [102]:
import numpy as np
data=scaler.inverse_transform(outputs_val,copy=None)
#recovered
#data
orig=np.linalg.norm(outputs_val,ord=1)
DiffData=data-returns_arr
NewNorm=np.linalg.norm(DiffData,ord=1)
error=NewNorm/orig 
error
#recovering the orignal dat

0.00047185753691870194